test file for nlp packages that predict race to see acuuracy,the following are the packages using 
 

3. raceBERT 
a transformer-based model for predict- ing race and ethnicity from character sequences in names, and an accom- panying python package. Using a transformer-based model trained on a U.S. Florida voter registration dataset1, the model predicts the likelihood of a name belonging to 5 U.S. census race categories (White, Black, His- panic, Asian & Pacific Islander, American Indian & Alaskan Native). 

according to the documentation, raceBERT achieves a 5% improvement over ethnicolr

Sources: https://arxiv.org/pdf/2112.03807.pdf 
https://pypi.org/project/racebert/  
https://huggingface.co/pparasurama/raceBERT 

4. predictrace --> must be done in R  
The model predicts the race of a surname or first name and the gender of a first name. This package uses U.S. Census data which says how many people of each race has a certain surname. For first name data, this package uses data from Tzioumis (2018). From this we can predict which race is mostly likely to have that surname or first name. The possible races are American Indian, Asian, Black, Hispanic, White, or two or more races. 

Source:  

https://cran.r-project.org/web/packages/predictrace/vignettes/Predict-race-of-surname.html 

The Prediction Process: 

owner ----> nlp ----> W: 70% NW: 30%    
geoid:059.... ----> W: 80% NW: 20%
            .
            .
            .
            .
geoid:059.... ----> W: 60% NW: 40%
            and use
        min & max scale? 
          ex.  
            1 -----> 40
        

In [12]:
import pandas as pd
from racebert import RaceBERT

# Load the dataset
mergent_true_df = pd.read_csv('../data/mergent_intellect_executives/sample.csv.xz')

# Extract first and last names
namesdf = pd.DataFrame(mergent_true_df, columns=['lastname', 'firstname'])
namesdf

,lastname,firstname
0,Maigue,Richard
1,Fatah,Ifrah
2,Abu-hamda,Eyad
3,Degraft,J
4,Padwo-audick,Dara
...,...,...
1062,Sharifzadeh Na,Lily
1063,Na,Lily
1064,Kim,Ronnie
1065,Shihadh,Naser


# using raceBert

In [13]:
# Instantiate the RaceBERT model
model = RaceBERT()

In [14]:
# Concatenate first and last names to form full names
namesdf['fullname'] = namesdf['firstname'] + ' ' + namesdf['lastname']

In [15]:
# Predict race for all names
predicted_race = model.predict_race(namesdf['fullname'])

# Add the predicted race to the original DataFrame
namesdf['predicted_race'] = predicted_race

In [16]:
# Create a new DataFrame to store the names and predicted races
result_df = pd.DataFrame({'fullname': namesdf['fullname'], 'predicted_race': predicted_race})

result_df


,fullname,predicted_race
0,Richard Maigue,"{'label': 'nh_white', 'score': 0.9404343962669..."
1,Ifrah Fatah,"{'label': 'nh_black', 'score': 0.4484985768795..."
2,Eyad Abu-hamda,"{'label': 'nh_white', 'score': 0.9295466542243..."
3,J Degraft,"{'label': 'nh_white', 'score': 0.9904143810272..."
4,Dara Padwo-audick,"{'label': 'nh_black', 'score': 0.8661506175994..."
...,...,...
1062,Lily Sharifzadeh Na,"{'label': 'api', 'score': 0.7320632338523865}"
1063,Lily Na,"{'label': 'api', 'score': 0.8636026978492737}"
1064,Ronnie Kim,"{'label': 'api', 'score': 0.8940410017967224}"
1065,Naser Shihadh,"{'label': 'nh_white', 'score': 0.8536479473114..."


In [17]:
# Split the 'predicted_race' column into 'label' and 'score' columns
result_df[['label', 'score']] = result_df['predicted_race'].apply(pd.Series)

# Drop the 'predicted_race' column
result_df.drop('predicted_race', axis=1, inplace=True)

In [18]:
result_df

,fullname,label,score
0,Richard Maigue,nh_white,0.940434
1,Ifrah Fatah,nh_black,0.448499
2,Eyad Abu-hamda,nh_white,0.929547
3,J Degraft,nh_white,0.990414
4,Dara Padwo-audick,nh_black,0.866151
...,...,...,...
1062,Lily Sharifzadeh Na,api,0.732063
1063,Lily Na,api,0.863603
1064,Ronnie Kim,api,0.894041
1065,Naser Shihadh,nh_white,0.853648


In [19]:
white = result_df[result_df['label'] == 'nh_white']
nonwhite = result_df[result_df['label'] != 'nh_white']

In [20]:
white

,fullname,label,score
0,Richard Maigue,nh_white,0.940434
2,Eyad Abu-hamda,nh_white,0.929547
3,J Degraft,nh_white,0.990414
5,Dara Padwoaudick,nh_white,0.932096
6,Christopher Smith,nh_white,0.827153
...,...,...,...
1059,Chris Moniz,nh_white,0.973491
1060,Terri Moreland,nh_white,0.730947
1061,Rick Smith,nh_white,0.911473
1065,Naser Shihadh,nh_white,0.853648


In [21]:
nonwhite

,fullname,label,score
1,Ifrah Fatah,nh_black,0.448499
4,Dara Padwo-audick,nh_black,0.866151
13,Alex Van,api,0.767921
19,Dinh Tao,api,0.982362
21,Yu Park,api,0.976893
...,...,...,...
1056,Buyanjargal Sandag,api,0.916281
1057,David Chan,api,0.911980
1062,Lily Sharifzadeh Na,api,0.732063
1063,Lily Na,api,0.863603


,fullname,label,score
1,Ifrah Fatah,nh_black,0.448499
4,Dara Padwo-audick,nh_black,0.866151
47,Jerome Baker,nh_black,0.696787
73,Del Alfred,nh_black,0.926769
81,Shawana Mufti,nh_black,0.883352
...,...,...,...
971,Norn Diop,nh_black,0.952390
975,Ashraf Adam-Eldien,nh_black,0.644479
977,Reginald Coleman,nh_black,0.919947
985,Kia Diaz,nh_black,0.521714
